In [ ]:
# Import required packages
from os import path 
import time
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot
from utils.dataset import create_dataset

In [ ]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=250_0000)
# build dimension reduction model
hnne, ds, loader = get_hnne_model(X, y)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer = get_flow_model()

In [ ]:
def train_step(model, batch):
    x, y = add_small_noise_to_batch(batch)
        
    loss = -flow(y).log_prob(x)  # -log p(x | y)
    loss = loss.mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [ ]:
step = 0
err_his = load_numpy(file_path=config.err_his_path)
loss_his = load_numpy(file_path=config.train_loss_his_path)
for ep in range(config.num_epochs):
    t = tqdm(loader)
    for batch in t:
        loss = train_step(model=flow, batch=batch)
        
        loss_his = np.concatenate((loss_his, [loss]))
        bar = {
            "loss": f"{np.round(loss, 3)}/{np.round(loss_his.mean(), 3)}",
            "ep": ep,
        }
        t.set_postfix(bar, refresh=True)

        step += 1
        if step % config.num_steps_save == 0:
            torch.save(flow.state_dict(), config.save_path)
            save_numpy(config.train_loss_his_path, loss_his)
            df, err = test_l2_err(config, step=step)
            print(df.describe())
            err_his = np.concatenate((err_his, [err]))
            save_numpy(config.err_his_path, err_his)

In [ ]:
df, err = test_l2_err(config, robot=panda, loader=loader, model=flow)
ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

In [ ]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(config, robot=panda, loader=loader, model=nflow)
ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

In [ ]:
def save_show_pose_data(config, num_data, num_samples, model=flow):
    batch = next(iter(loader))
    x, y = add_small_noise_to_batch(batch, eval=True)
    assert num_data < len(x)
    
    x_hats = np.array([])
    pidxs = np.array([])
    errs = np.array([])
    log_probs = np.array([])
    rand = np.random.randint(low=0, high=len(x), size=num_data)
    
    for nd in rand:
        x_hat = model(y[nd]).sample((num_samples,))
        log_prob = model(y[nd]).log_prob(x_hat)
        
        x_hat = x_hat.detach().cpu().numpy()
        log_prob = -log_prob.detach().cpu().numpy()
        target = y[nd].detach().cpu().numpy()
        # ee_pos = ee_pos * (ds.targets_max - ds.targets_min) + ds.targets_min
        ee_pos = target[:3]
        
        for q in x_hat:
            err = panda.dist_fk(q=q, ee_pos=ee_pos)
            errs = np.concatenate((errs, [err]))
        x_hats = np.concatenate((x_hats, x_hat.reshape(-1)))
        pidx = target[3:-1]
        pidx = np.tile(pidx, (num_samples, 1))

        pidxs = np.concatenate((pidxs, pidx.reshape(-1)))
        log_probs = np.concatenate((log_probs, log_prob))

    x_hats = x_hats.reshape((-1, panda.dof))
    pidxs = pidxs.reshape((len(x_hats), -1))
    

    save_numpy(config.show_pose_features_path, x_hats)
    save_numpy(config.show_pose_pidxs_path, pidxs)
    save_numpy(config.show_pose_errs_path, errs)
    save_numpy(config.show_pose_log_probs_path, log_probs)
    
    print('Save pose successfully')

In [ ]:
save_show_pose_data(config, num_data=5, num_samples=10, model=nflow)

In [ ]:
def inside_same_pidx():
    x_hats = load_numpy(file_path=config.show_pose_features_path)
    pidxs = load_numpy(file_path=config.show_pose_pidxs_path)
    
    if len(x_hats) == 0:
        raise ValueError("lack show pose data") 
    
    pre_pidx = None
    qs = np.array([])
    for q, pidx in zip(x_hats, pidxs):
        if pre_pidx is None or np.array_equal(pre_pidx, pidx):
            qs = np.concatenate((qs, q))
        else:
            break
        pre_pidx = pidx
    qs = qs.reshape((-1, panda.dof))
    for q in qs:
        panda.plot(q, q)

In [ ]:
inside_same_pidx()